In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import math
import numpy as np
import pandas as pd
import glob
import time

from sklearn import linear_model, model_selection, metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import OneClassSVM
from sklearn import model_selection, metrics
from scipy import stats # for statistical tests

import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive

plt.style.use('seaborn')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# Get the dataset
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Sentiment Analysis

In [6]:
train = pd.read_pickle('gdrive/My Drive/sent_train.pkl')
train.head()

,index,feature,raw_clause,clause_vector,clause_polarity
0,154,review,Thank you for leaving a review,"[[-0.49886, 0.76602, 0.89751, -0.78547, -0.685...",0.00
0,154,dinner,\r\n\r\n Restaurant Ibid is now re - opened fo...,"[[-0.54264, 0.41476, 1.0322, -0.40244, 0.46691...",0.00
0,154,Mondays,\r\n\r\n Restaurant Ibid is now re - opened fo...,"[[-0.54264, 0.41476, 1.0322, -0.40244, 0.46691...",0.00
0,154,forwards,\r\n\r\n Looking forwards to hosting you,"[[0.29756, 0.70983, 0.65643, -1.3487, -0.41072...",0.00
0,154,reservations,reservations are highly recommended,"[[-0.13411, -0.39639, 0.64715, 0.38583, -0.225...",0.16


In [7]:
test = pd.read_pickle('gdrive/My Drive/sent_test.pkl')
test.head()

,index,feature,raw_clause,clause_vector,clause_polarity
0,176,burger,I have nt had such a delicious burger is a while,"[[0.15711, 0.65606, 0.0021149, -0.65144, -0.28...",0.5
0,176,while,I have nt had such a delicious burger is a while,"[[0.15711, 0.65606, 0.0021149, -0.65144, -0.28...",0.5
0,176,burgers,Great burgers at reasonable prices,"[[-0.42404, 0.0074403, -0.4665, -0.42496, -0.5...",0.5
0,176,prices,Great burgers at reasonable prices,"[[-0.42404, 0.0074403, -0.4665, -0.42496, -0.5...",0.5
0,176,Onion,Onion rings are crispy awesome too,"[[0.65609, 0.03754, -0.14359, -0.47495, 0.8703...",1.0


In [8]:
val = pd.read_pickle('gdrive/My Drive/sent_validate.pkl')
val.head()

,index,feature,raw_clause,clause_vector,clause_polarity
0,296,friends,Outdoor is kinda hot to chill with friends,"[[-0.54264, 0.41476, 1.0322, -0.40244, 0.46691...",0.25
0,296,Customer,Customer service good,"[[-0.4224, -0.13313, -0.41418, -0.23677, 0.190...",0.70
0,296,service,Customer service good,"[[-0.4224, -0.13313, -0.41418, -0.23677, 0.190...",0.70
0,112,review,Thank you for leaving a review,"[[-0.49886, 0.76602, 0.89751, -0.78547, -0.685...",0.00
0,112,dinner,\r\n\r\n Restaurant Ibid is now re - opened fo...,"[[-0.54264, 0.41476, 1.0322, -0.40244, 0.46691...",0.00


In [9]:
to_pred = pd.read_pickle('gdrive/My Drive/testing_sent_df.pkl')
to_pred.head()

,restaurant,stars,date,feat_index,index,feature,category,raw_clause,clause_vector,clause_polarity
0,Gravy_Restaurant,5 stars,2 days ago,0,There are lots of Indian restaurant at Tanjong...,lots,price,There are lots of Indian restaurant at Tanjong...,"[[-0.51533, 0.83186, 0.22457, -0.73865, 0.1871...",0.0
0,Gravy_Restaurant,5 stars,2 days ago,1,There are lots of Indian restaurant at Tanjong...,restaurant,ambience,There are lots of Indian restaurant at Tanjong...,"[[-0.51533, 0.83186, 0.22457, -0.73865, 0.1871...",0.0
0,Gravy_Restaurant,5 stars,2 days ago,2,There are lots of Indian restaurant at Tanjong...,time,price,Second time dining out since the dining restri...,"[[-0.024221, -0.034855, 0.3571, -0.02155, -0.5...",0.0
0,Gravy_Restaurant,5 stars,2 days ago,3,There are lots of Indian restaurant at Tanjong...,dining,ambience,Second time dining out since the dining restri...,"[[-0.024221, -0.034855, 0.3571, -0.02155, -0.5...",0.0
0,Gravy_Restaurant,5 stars,2 days ago,4,There are lots of Indian restaurant at Tanjong...,restrictions,price,Second time dining out since the dining restri...,"[[-0.024221, -0.034855, 0.3571, -0.02155, -0.5...",0.0


In [12]:
number_of_most_frequent_words=10000
max_len=500

In [13]:
X_train = train['raw_clause']
y_train = train['clause_polarity']

In [14]:
X_val = val['raw_clause']
y_val = val['clause_polarity']

In [15]:
X_test = test['raw_clause']
y_test= test['clause_polarity']

In [16]:
X_tp = to_pred['raw_clause']
y_tp = to_pred['clause_polarity']

In [17]:
X_val

0           Outdoor is kinda hot to chill with friends
0                                Customer service good
0                                Customer service good
0                       Thank you for leaving a review
0    \r\n\r\n Restaurant Ibid is now re - opened fo...
                           ...                        
0                                 this is steak heaven
0    was whisked away to a unforgettable experience...
0    was whisked away to a unforgettable experience...
0                            go for the early cut menu
0                            go for the early cut menu
Name: raw_clause, Length: 2902, dtype: object

In [18]:
X_train

0                       Thank you for leaving a review
0    \r\n\r\n Restaurant Ibid is now re - opened fo...
0    \r\n\r\n Restaurant Ibid is now re - opened fo...
0             \r\n\r\n Looking forwards to hosting you
0                  reservations are highly recommended
                           ...                        
0                                   Came in vegetarian
0                                           The sauces
0                   came out wanting to eat more steak
0                                          service and
0                                                sides
Name: raw_clause, Length: 8909, dtype: object

In [19]:
X_train = train['raw_clause'].to_numpy()
X_val = val['raw_clause'].to_numpy()
X_test = test['raw_clause'].to_numpy()
X_tp = to_pred['raw_clause'].to_numpy()

In [21]:
from keras.preprocessing.text import Tokenizer

In [22]:
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(X_train)

In [23]:
X_train_tokens = tokenizer_obj.texts_to_sequences(X_train)
X_val_tokens = tokenizer_obj.texts_to_sequences(X_val)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test)
X_tp_tokens = tokenizer_obj.texts_to_sequences(X_tp)

In [24]:
max_length = max([len(s.split()) for s in X_train])

In [25]:
vocab_size = len(tokenizer_obj.word_index) + 1

In [27]:
from keras.preprocessing.sequence import pad_sequences

In [28]:
X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, padding='post')
X_val_pad = pad_sequences(X_val_tokens, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length, padding='post')
X_tp_pad = pad_sequences(X_tp_tokens, maxlen=max_length, padding='post')

In [29]:
from keras.callbacks import ReduceLROnPlateau,EarlyStopping

In [30]:
rlr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr = 0.00001,verbose=1)
early = EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)

In [33]:
from tensorflow.keras import Sequential
from keras.layers import GRU,Embedding, Flatten, Conv1D, GlobalMaxPooling1D, Dropout, Dense, Input

EMBEDDING_DIM = 100
model = Sequential()
e = Embedding(vocab_size, EMBEDDING_DIM, input_length=max_length)
model.add(e)
model.add(Conv1D(1024, 3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dense(1))
model.compile(loss="mse", optimizer="adam", metrics=['accuracy'])
model.fit(X_train_pad, y_train, validation_data=(X_val_pad, y_val), epochs=100, batch_size=32, verbose=2, callbacks=[rlr, early])

Epoch 1/100
279/279 - 23s - loss: 0.0410 - accuracy: 0.4614 - val_loss: 0.0222 - val_accuracy: 0.4635 - lr: 0.0010 - 23s/epoch - 83ms/step
Epoch 2/100
279/279 - 22s - loss: 0.0092 - accuracy: 0.4730 - val_loss: 0.0154 - val_accuracy: 0.4673 - lr: 0.0010 - 22s/epoch - 78ms/step
Epoch 3/100
279/279 - 22s - loss: 0.0044 - accuracy: 0.4736 - val_loss: 0.0133 - val_accuracy: 0.4666 - lr: 0.0010 - 22s/epoch - 78ms/step
Epoch 4/100
279/279 - 22s - loss: 0.0029 - accuracy: 0.4736 - val_loss: 0.0118 - val_accuracy: 0.4669 - lr: 0.0010 - 22s/epoch - 78ms/step
Epoch 5/100
279/279 - 22s - loss: 0.0022 - accuracy: 0.4736 - val_loss: 0.0114 - val_accuracy: 0.4669 - lr: 0.0010 - 22s/epoch - 78ms/step
Epoch 6/100
279/279 - 22s - loss: 0.0019 - accuracy: 0.4736 - val_loss: 0.0116 - val_accuracy: 0.4669 - lr: 0.0010 - 22s/epoch - 78ms/step
Epoch 7/100
279/279 - 22s - loss: 0.0017 - accuracy: 0.4736 - val_loss: 0.0112 - val_accuracy: 0.4662 - lr: 0.0010 - 22s/epoch - 79ms/step
Epoch 8/100
279/279 - 22s -

In [34]:
pred = model.predict(X_test_pad)

In [35]:
pred

array([[0.5979684 ],
       [0.5979684 ],
       [0.5543965 ],
       ...,
       [0.08639959],
       [0.08639959],
       [0.08639959]], dtype=float32)

In [36]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, pred)

0.017489370575872656

In [37]:
final = test['feature']

In [38]:
pred.flatten()

array([0.5979684 , 0.5979684 , 0.5543965 , ..., 0.08639959, 0.08639959,
       0.08639959], dtype=float32)

In [39]:
final.to_numpy()

array(['burger', 'while', 'burgers', ..., 'couple', 'dozen', 'oysters'],
      dtype=object)

In [40]:
pred = pd.DataFrame(pred)
final = pd.DataFrame(final)

In [41]:
result4 = final

In [42]:
result4.index = pred.index

In [43]:
result4['pred'] = pred

In [50]:
result4

,feature,pred
0,burger,0.597968
1,while,0.597968
2,burgers,0.554397
3,prices,0.554397
4,Onion,0.934958
...,...,...
2973,service,1.043494
2974,porterhouse,-0.026459
2975,couple,0.086400
2976,dozen,0.086400


In [49]:
y_test

,clause_polarity
0,0.5
0,0.5
0,0.5
0,0.5
0,1.0
...,...
0,1.0
0,0.0
0,0.0
0,0.0


In [ ]:
from google.colab import files
result4.to_csv('preds4222.csv', index=False) 
files.download('preds4222.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
y_test.to_csv('ytest.csv', index=False) 
files.download('ytest.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [67]:
pred = model.predict(X_tp_pad)

In [68]:
pred

array([[ 0.02025438],
       [ 0.02025438],
       [-0.00995666],
       ...,
       [ 0.27647686],
       [ 0.27647686],
       [ 0.7429442 ]], dtype=float32)

In [69]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_tp, pred)

0.015364141044826385

In [70]:
final = to_pred['feature']

In [71]:
pred.flatten()

array([ 0.02025438,  0.02025438, -0.00995666, ...,  0.27647686,
        0.27647686,  0.7429442 ], dtype=float32)

In [72]:
final.to_numpy()

array(['lots', 'restaurant', 'time', ..., 'tables', 'restaurant', 'menu'],
      dtype=object)

In [73]:
pred = pd.DataFrame(pred)
final = pd.DataFrame(final)

In [74]:
result_new = final

In [75]:
result_new.index = pred.index

In [76]:
result_new['pred'] = pred

In [77]:
result_new

,feature,pred
0,lots,0.020254
1,restaurant,0.020254
2,time,-0.009957
3,dining,-0.009957
4,restrictions,-0.009957
...,...,...
4225,food,0.732610
4226,hospitality,0.276477
4227,tables,0.276477
4228,restaurant,0.276477


In [78]:
y_tp

0    0.000000
0    0.000000
0    0.000000
0    0.000000
0    0.000000
       ...   
0    0.400000
0    0.333333
0    0.333333
0    0.333333
0    0.400000
Name: clause_polarity, Length: 4230, dtype: float64

In [79]:
from google.colab import files
result_new.to_csv('preds4222_newtest.csv', index=False) 
files.download('preds4222_newtest.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [80]:
from google.colab import files
y_tp.to_csv('ynew.csv', index=False) 
files.download('ynew.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [83]:
rlr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr = 0.00001,verbose=1)
early = EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)

In [84]:
from tensorflow.keras import Sequential
from keras.layers import GRU,Embedding, Flatten, Conv1D, GlobalMaxPooling1D, Dropout, Dense, Input

EMBEDDING_DIM = 100
model = Sequential()
e = Embedding(vocab_size, EMBEDDING_DIM, input_length=max_length)
model.add(e)
model.add(Conv1D(1024, 3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dense(1))
model.compile(loss="mse", optimizer="adam", metrics=['accuracy'])
model.fit(X_train_pad, y_train, validation_data=(X_val_pad, y_val), epochs=100, batch_size=32, verbose=2, callbacks=[rlr, early])

Epoch 1/100
279/279 - 23s - loss: 0.0416 - accuracy: 0.4609 - val_loss: 0.0230 - val_accuracy: 0.4649 - lr: 0.0010 - 23s/epoch - 83ms/step
Epoch 2/100
279/279 - 23s - loss: 0.0094 - accuracy: 0.4731 - val_loss: 0.0152 - val_accuracy: 0.4669 - lr: 0.0010 - 23s/epoch - 81ms/step
Epoch 3/100
279/279 - 23s - loss: 0.0044 - accuracy: 0.4736 - val_loss: 0.0135 - val_accuracy: 0.4666 - lr: 0.0010 - 23s/epoch - 81ms/step
Epoch 4/100
279/279 - 22s - loss: 0.0027 - accuracy: 0.4736 - val_loss: 0.0124 - val_accuracy: 0.4669 - lr: 0.0010 - 22s/epoch - 80ms/step
Epoch 5/100
279/279 - 22s - loss: 0.0022 - accuracy: 0.4736 - val_loss: 0.0122 - val_accuracy: 0.4666 - lr: 0.0010 - 22s/epoch - 80ms/step
Epoch 6/100
279/279 - 22s - loss: 0.0019 - accuracy: 0.4736 - val_loss: 0.0115 - val_accuracy: 0.4673 - lr: 0.0010 - 22s/epoch - 80ms/step
Epoch 7/100
279/279 - 22s - loss: 0.0015 - accuracy: 0.4736 - val_loss: 0.0113 - val_accuracy: 0.4673 - lr: 0.0010 - 22s/epoch - 79ms/step
Epoch 8/100
279/279 - 22s -

In [85]:
pred = model.predict(X_test_pad)

In [86]:
pred

array([[ 0.49301174],
       [ 0.49301174],
       [ 0.47084165],
       ...,
       [-0.00269047],
       [-0.00269047],
       [-0.00269047]], dtype=float32)

In [87]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, pred)

0.010722351682833123

In [88]:
final = test['feature']

In [89]:
pred.flatten()

array([ 0.49301174,  0.49301174,  0.47084165, ..., -0.00269047,
       -0.00269047, -0.00269047], dtype=float32)

In [90]:
final.to_numpy()

array(['burger', 'while', 'burgers', ..., 'couple', 'dozen', 'oysters'],
      dtype=object)

In [91]:
pred = pd.DataFrame(pred)
final = pd.DataFrame(final)

In [92]:
result4 = final

In [93]:
result4.index = pred.index

In [94]:
result4['pred'] = pred

In [95]:
result4

,feature,pred
0,burger,0.493012
1,while,0.493012
2,burgers,0.470842
3,prices,0.470842
4,Onion,0.945860
...,...,...
2973,service,0.985742
2974,porterhouse,-0.003645
2975,couple,-0.002690
2976,dozen,-0.002690


In [96]:
y_test

,clause_polarity
0,0.5
0,0.5
0,0.5
0,0.5
0,1.0
...,...
0,1.0
0,0.0
0,0.0
0,0.0


In [ ]:
from google.colab import files
result4.to_csv('preds4222.csv', index=False) 
files.download('preds4222.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
y_test.to_csv('ytest.csv', index=False) 
files.download('ytest.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [97]:
pred = model.predict(X_tp_pad)

In [98]:
pred

array([[-0.00453277],
       [-0.00453277],
       [-0.00480882],
       ...,
       [ 0.24750957],
       [ 0.24750957],
       [ 0.7817584 ]], dtype=float32)

In [99]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_tp, pred)

0.009499475102893474

In [100]:
final = to_pred['feature']

In [101]:
pred.flatten()

array([-0.00453277, -0.00453277, -0.00480882, ...,  0.24750957,
        0.24750957,  0.7817584 ], dtype=float32)

In [102]:
final.to_numpy()

array(['lots', 'restaurant', 'time', ..., 'tables', 'restaurant', 'menu'],
      dtype=object)

In [103]:
pred = pd.DataFrame(pred)
final = pd.DataFrame(final)

In [104]:
result_new = final

In [105]:
result_new.index = pred.index

In [106]:
result_new['pred'] = pred

In [107]:
result_new

,feature,pred
0,lots,-0.004533
1,restaurant,-0.004533
2,time,-0.004809
3,dining,-0.004809
4,restrictions,-0.004809
...,...,...
4225,food,0.784040
4226,hospitality,0.247510
4227,tables,0.247510
4228,restaurant,0.247510


In [108]:
y_tp

0    0.000000
0    0.000000
0    0.000000
0    0.000000
0    0.000000
       ...   
0    0.400000
0    0.333333
0    0.333333
0    0.333333
0    0.400000
Name: clause_polarity, Length: 4230, dtype: float64

In [112]:
to_pred

,restaurant,stars,date,feat_index,index,feature,category,raw_clause,clause_vector,clause_polarity
0,Gravy_Restaurant,5 stars,2 days ago,0,There are lots of Indian restaurant at Tanjong...,lots,price,There are lots of Indian restaurant at Tanjong...,"[[-0.51533, 0.83186, 0.22457, -0.73865, 0.1871...",0.000000
0,Gravy_Restaurant,5 stars,2 days ago,1,There are lots of Indian restaurant at Tanjong...,restaurant,ambience,There are lots of Indian restaurant at Tanjong...,"[[-0.51533, 0.83186, 0.22457, -0.73865, 0.1871...",0.000000
0,Gravy_Restaurant,5 stars,2 days ago,2,There are lots of Indian restaurant at Tanjong...,time,price,Second time dining out since the dining restri...,"[[-0.024221, -0.034855, 0.3571, -0.02155, -0.5...",0.000000
0,Gravy_Restaurant,5 stars,2 days ago,3,There are lots of Indian restaurant at Tanjong...,dining,ambience,Second time dining out since the dining restri...,"[[-0.024221, -0.034855, 0.3571, -0.02155, -0.5...",0.000000
0,Gravy_Restaurant,5 stars,2 days ago,4,There are lots of Indian restaurant at Tanjong...,restrictions,price,Second time dining out since the dining restri...,"[[-0.024221, -0.034855, 0.3571, -0.02155, -0.5...",0.000000
...,...,...,...,...,...,...,...,...,...,...
0,Comida_Mexicana,1 star,4 years ago,0,Great Mexican food! Great hospitality within t...,food,food,Great Mexican food,"[[-0.16486, 0.91997, 0.22737, -0.49031, -0.001...",0.400000
0,Comida_Mexicana,1 star,4 years ago,1,Great Mexican food! Great hospitality within t...,hospitality,price,Great hospitality within the few tables availa...,"[[0.23471, 0.097874, 0.049255, 0.42204, -0.253...",0.333333
0,Comida_Mexicana,1 star,4 years ago,2,Great Mexican food! Great hospitality within t...,tables,ambience,Great hospitality within the few tables availa...,"[[0.23471, 0.097874, 0.049255, 0.42204, -0.253...",0.333333
0,Comida_Mexicana,1 star,4 years ago,3,Great Mexican food! Great hospitality within t...,restaurant,ambience,Great hospitality within the few tables availa...,"[[0.23471, 0.097874, 0.049255, 0.42204, -0.253...",0.333333


In [119]:
place = to_pred['restaurant']
category = to_pred['category']

In [121]:
place = pd.DataFrame(place)

In [122]:
place.index = pred.index

In [123]:
result_new['place'] = place

In [126]:
category = pd.DataFrame(category)
category.index = pred.index
result_new['category'] = category

In [127]:
result_new

,feature,pred,place,category
0,lots,-0.004533,Gravy_Restaurant,price
1,restaurant,-0.004533,Gravy_Restaurant,ambience
2,time,-0.004809,Gravy_Restaurant,price
3,dining,-0.004809,Gravy_Restaurant,ambience
4,restrictions,-0.004809,Gravy_Restaurant,price
...,...,...,...,...
4225,food,0.784040,Comida_Mexicana,food
4226,hospitality,0.247510,Comida_Mexicana,price
4227,tables,0.247510,Comida_Mexicana,ambience
4228,restaurant,0.247510,Comida_Mexicana,ambience


In [128]:
from google.colab import files
result_new.to_csv('preds4222_newtest2.csv', index=False) 
files.download('preds4222_newtest2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
y_tp.to_csv('ynew.csv', index=False) 
files.download('ynew.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>